## Installing dependencies

In [1]:
!pip install omegaconf --user
!pip install nemo --user
!pip install hydra --user

     ---------------------------------------- 0.0/79.5 kB ? eta -:--:--
     ---------------------------------------- 79.5/79.5 kB 4.6 MB/s eta 0:00:00
     ---------------------------------------- 0.0/117.0 kB ? eta -:--:--
     -------------------------------------- 117.0/117.0 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/151.7 kB ? eta -:--:--
     -------------------------------------- 151.7/151.7 kB 9.4 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144578 sha256=4df8883cc81505bd846bdc015f0f3510b9c1b9acde10e8e22ab2406d3c948469
  Stored in directory: c:\users\mahdj\appdata\local\pip\cache\wheels\12\93\dd\1f6a127edc45659556564c5730f6d4e300888f4bca2d4c5a88
Successfully built antlr4-python3-runtime
  Using cached Hydra-2.5.tar.gz (82 kB)
  Preparing metadata (setup.py): started
  Pre

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [9 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-310
      copying src\hydra.py -> build\lib.win-amd64-cpython-310
      running build_ext
      building '_hydra' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for hydra
  error: subprocess-exited-with-error
  
  × Running setup.py install for hydra did not run successfully.
  │ exit code: 1
  ╰─> [11 lines of output]
      running install
      C:\Users\mahdj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-p

In [2]:
import site
site.addsitedir(site.getusersitepackages())
p=site.getusersitepackages()
!ls $p

'ls' is not recognized as an internal or external command,
operable program or batch file.


## Preprocessing

### Preapare Data

In [3]:
# Import useful utilities for grading
import os
import glob
from omegaconf import OmegaConf

def get_latest_model():  
    nemo_model_paths = glob.glob('nemo_experiments/TextClassification/*/checkpoints/*.nemo')
    # Sort newest first
    nemo_model_paths.sort(reverse=True)
    return nemo_model_paths[0]

In [5]:
DATA_DIR = './data'
print(os.listdir(DATA_DIR))

['eva.tsv', 'eva_nemo_format.tsv', 'sample.csv', 'test.csv', 'train.csv', 'train.tsv', 'train_clean.csv', 'train_nemo_format.tsv']


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
data=pd.read_csv("./data/train.csv",index_col=False)
data=data.drop(columns='ID',axis=1)
X = data.drop(columns=["label"])
y = data["label"]
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train=pd.DataFrame(X_train,y_train)
data.to_csv('./data/train.tsv', sep='\t', index=False)
X_test.to_csv('./data/eva.tsv', sep='\t', index=False)

In [9]:
!sed 1d $DATA_DIR/train.tsv > $DATA_DIR/train_nemo_format.tsv
!sed 1d $DATA_DIR/eva.tsv > $DATA_DIR/eva_nemo_format.tsv

'sed' is not recognized as an internal or external command,
operable program or batch file.
'sed' is not recognized as an internal or external command,
operable program or batch file.


In [10]:
# check your work
print("*****\ntrain_nemo_format.tsv sample\n*****")
!head -n 3 $DATA_DIR/train_nemo_format.tsv
print("*****\ntrain_nemo_format.tsv sample\n*****")
!head -n 3 $DATA_DIR/eva_nemo_format.tsv

*****
train_nemo_format.tsv sample
*****
*****
train_nemo_format.tsv sample
*****


'head' is not recognized as an internal or external command,
operable program or batch file.
'head' is not recognized as an internal or external command,
operable program or batch file.


### preparing Model Configuration

In [11]:
# Take a look at the default model portion of the config file
CONFIG_DIR = "../Lib/Models/Classifiers/DL/NLP/text_classification/conf"
CONFIG_FILE = "text_classification_config.yaml"

config = OmegaConf.load(CONFIG_DIR + "/" + CONFIG_FILE)
print(OmegaConf.to_yaml(config.model))

nemo_path: text_classification_model.nemo
tokenizer:
  tokenizer_name: ${model.language_model.pretrained_model_name}
  vocab_file: null
  tokenizer_model: null
  special_tokens: null
language_model:
  pretrained_model_name: bert-base-uncased
  lm_checkpoint: null
  config_file: null
  config: null
classifier_head:
  num_output_layers: 2
  fc_dropout: 0.1
class_labels:
  class_labels_file: null
dataset:
  num_classes: ???
  do_lower_case: false
  max_seq_length: 256
  class_balancing: null
  use_cache: false
train_ds:
  file_path: null
  batch_size: 64
  shuffle: true
  num_samples: -1
  num_workers: 3
  drop_last: false
  pin_memory: false
validation_ds:
  file_path: null
  batch_size: 64
  shuffle: false
  num_samples: -1
  num_workers: 3
  drop_last: false
  pin_memory: false
test_ds:
  file_path: null
  batch_size: 64
  shuffle: false
  num_samples: -1
  num_workers: 3
  drop_last: false
  pin_memory: false
optim:
  name: adam
  lr: 2.0e-05
  betas:
  - 0.9
  - 0.999
  weight_decay:

In [12]:
# See what BERT-like language models are available
from nemo.collections import nlp as nemo_nlp
nemo_nlp.modules.get_pretrained_lm_models_list()

ModuleNotFoundError: No module named 'nemo'

In [ ]:
# set the values
NUM_CLASSES = 3
MAX_SEQ_LENGTH = 256
BATCH_SIZE = 32
PATH_TO_TRAIN_FILE = "./data/train_nemo_format.tsv"
PATH_TO_VAL_FILE = "./data/eva_nemo_format.tsv"
PRETRAINED_MODEL_NAME = 'bert-base-uncased' # change as desired
LR = 2e-5 # change as desired

### prepare trainer

In [ ]:
# set the values
MAX_EPOCHS = 11
AMP_LEVEL = 'O1'
PRECISION = 16

## Data Mining

In [ ]:
%%time
# Run the training script, overriding the config values in the command line
TC_DIR = "../Lib/Models/Classifiers/DL/NLP/text_classification"

!python $TC_DIR/text_classification_with_bert.py \
        model.dataset.num_classes=$NUM_CLASSES \
        model.dataset.max_seq_length=$MAX_SEQ_LENGTH \
        model.train_ds.file_path=$PATH_TO_TRAIN_FILE \
        model.validation_ds.file_path=$PATH_TO_VAL_FILE \
        model.infer_samples=[] \
        trainer.max_epochs=$MAX_EPOCHS \
        model.language_model.pretrained_model_name=$PRETRAINED_MODEL_NAME \
        trainer.amp_level=$AMP_LEVEL \
        trainer.precision=$PRECISION \
        model.train_ds.batch_size=$BATCH_SIZE \
        model.validation_ds.batch_size=$BATCH_SIZE\
        model.optim.lr=$LR

  File "../Lib/Models/Classifiers/DL/NLP/text_classification/text_classification_with_bert.py", line 109
    def main(cfg: DictConfig) -> None:
                ^
SyntaxError: invalid syntax
CPU times: user 909 µs, sys: 15.6 ms, total: 16.5 ms
Wall time: 183 ms


## Evaluation

## Prediction
